#### [Santander Customer Satisfaction](https://www.kaggle.com/c/santander-customer-satisfaction/data)
桑坦德客户满意度：哪些客户是满意的客户？  
您将获得包含大量数字变量的匿名数据集。“TARGET”列是要预测的变量。1表示不满意的客户，0表示满意的客户。  
任务是预测测试集中每个客户是不满意客户的概率。  
 
#### 文件说明
train.csv - 包括目标的训练集  
test.csv - 没有目标的测试集  
sample_submission.csv - 格式正确的示例提交文件

# 1、依赖包的导入

In [1]:
# coding:utf-8
# 在python2 中导入未来的支持的语言特征中division(精确除法)，即from __future__ import division ，当我们在程序中没有导入该特征时，"/"操作符执行的只能是整除，也就是取整数，只有当我们导入division(精确算法)以后，"/"执行的才是精确算法。
from __future__ import division
import numpy as np
import pandas as pd
from scipy import stats

# 2、查看数据相关信息

In [3]:
df = pd.read_csv('./data/train.csv')
print(df.shape)
print(df.info())
#print(df.describe())  属性数量太多，不宜使用
df.head()

(76020, 371)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB
None


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


# 3、分离特征和目标值，并删除无用的特征
Python制作数据分析工具   
视频地址：https://www.imooc.com/video/16378   
推荐指数：*  

In [4]:
label = df['TARGET']
df = df.drop(['ID', 'TARGET'], axis=1)   # 除了ID，还有370个特征（好多）

# 4、特征基础分析：不同值的数量、缺失值、零值

In [5]:
##1.Basic Analysis##
#(1)Missing Value#
missSet = [np.nan, 9999999999, -999999]

#(2)Count distinct#
len(df.iloc[:, 0].unique())
count_un = df.iloc[:,0:3].apply(lambda x:len(x.unique()))

#(3)Zero Values#
np.sum(df.iloc[:0] == 0)
count_zero = df.iloc[:, 0:3].apply(lambda x:np.sum(x == 0))

print(count_un)
print(count_zero)

var3                  208
var15                 100
imp_ent_var16_ult1    596
dtype: int64
var3                     75
var15                     0
imp_ent_var16_ult1    72301
dtype: int64


# 5、均值/中位数
没有去除缺失值之前的均值和中位数很低

In [ ]:
#(4)Mean Values#
np.mean(df.iloc[:, 0]) # 没有去除缺失值之前的均值很低

df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)] # 去除缺失值
np.mean(df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)]) # 去除缺失值后的均值计算

# 6、众数
numpy没有众数函数，scipy的stats里面有

In [ ]:
#(6)Mode Values#
df_mode = df.iloc[:,0:3].apply(lambda x: stats.mode(x[~np.isin(x, missSet)])[0][0])

#(7)Mode Percentage#
df_mode_count = df.iloc[:,0:3].apply(lambda x: stats.mode(x[~np.isin(x, missSet)])[1][0]

df_mode_perct = df_mode_count / df.shape[0]

# 7、最大最小值

In [ ]:
#(8)Min Values#
np.min(df.iloc[:,0])

df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)]
np.min(df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)])

# 8、分位点
思考：为啥不用describe函数

In [6]:
#(10)quantile values
np.percentile(df.iloc[:,0], (1,5,25,50,75,95,99))

df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)]
np.percentile(df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)], (1,5,25,50,75,95,99))

array([ 2.,  2.,  2.,  2.,  2.,  2., 11.])

In [2]:
print(np.__version__)

1.15.2


# 9、频数

In [ ]:
#(11)Frequent Values
df.iloc[:,0].value_counts().iloc[0:5,]

df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)]
df.iloc[:,0][~np.isin(df.iloc[:,0], missSet)].value_counts()[0:5,]

In [ ]:
当返回值不只是一个的时候，apply函数就不适用了
pd.Dataframe.value_counts().iloc[0:5,].index.values

top5
字典，index是变量值，值是频数

# 10、缺失值

In [ ]:
#(12)Miss Values
np.sum(np.isin(df.iloc[:,0], missSet))
df_miss = df.iloc[:,0:3].apply(lambda x:np.sum(np.isin(x, missSet)))

# 11、工具整合
把批处理写成一个函数。   
数组可以直接使用to_frame()函数转换成Dataframe，然后用contact函数连接。   
简直就是实现了一遍describe函数。